In [53]:
from glob import glob
from PIL import Image
import numpy as np
from os.path import basename
import fnmatch
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

# Preparation of Images for our DataSet

In [54]:
#Path of the files:
path = "/Users/MariaNuila/Desktop/CECS406/IDC_regular_ps50_idx5/*/*/*.png"
width, height = 50, 50
pictures = glob(path)
#Separate images based on classification
nonIDC =  fnmatch.filter(pictures, '*class0.png')
IDC = fnmatch.filter(pictures, '*class1.png')


In [55]:
#Create training and testing data set from pictures
x = []
y = [] #labels
for pic in pictures[0:100000]:
    image = cv2.imread(pic)
    x.append(cv2.resize(image, (width, height), interpolation=cv2.INTER_CUBIC))
    if pic in nonIDC:
        y.append(0)
    if pic in IDC:
        y.append(1)
#Place in dataframe:
df = pd.DataFrame()
df["images"] = x
df["labels"] = y

#vectorize and normalize data
x=np.array(x)
x=x/255.0
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2)

#Convert our training and testing data into np arrays:
xtrain_npArray = np.array(X_train)
xtest_npArray = np.array(X_test)
ytrain_npArray = np.array(Y_train)
ytest_npArray = np.array(Y_test)

In [56]:
input_shape = (50,50,3)
# convert class vectors to binary class matrices
ytrain_npArray = keras.utils.to_categorical(ytrain_npArray, num_classes)
ytest_npArray = keras.utils.to_categorical(ytest_npArray, num_classes)

In [57]:
#Specify Data training information: 
batch_size = 128
num_classes = 2
epochs = 50

# Build Fine tuned VGG16 Model From Keras

In [58]:
vgg16_model = keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

In [59]:
vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 50, 50, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 50, 50, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 50, 50, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 25, 25, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 25, 25, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 25, 25, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [60]:
# Make our fine-tuned model sequential for easier processing
fine_tuned = Sequential()
#Add vgg layers to our model
for layers in vgg16_model.layers:
    fine_tuned.add(layers)

In [61]:
#Make the previous VGG model frozen with its current weights
for layers in fine_tuned.layers:
    layers.trainable = False

In [62]:
#Now add our fully connected layer:
fine_tuned.add(Flatten())
fine_tuned.add(Dense(64, activation='relu'))
fine_tuned.add(Dense(64, activation='relu'))
fine_tuned.add(Dropout(0.5))
fine_tuned.add(Dense(num_classes, activation='softmax'))

In [63]:
fine_tuned.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 50, 50, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 50, 50, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 50, 50, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 25, 25, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 25, 25, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 25, 25, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

# Train Fine-Tuned Model 

In [66]:
#use Adam optimizer based on stochastic optimization (Adam):
fine_tuned.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(lr=0.0001),
                  metrics=['accuracy'])

In [ ]:
fine_tuned.fit(xtrain_npArray, ytrain_npArray,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(xtest_npArray, ytest_npArray))

Train on 80000 samples, validate on 20000 samples
Epoch 1/50
80000/80000 [==============================] - 1343s 17ms/step - loss: 0.4179 - acc: 0.8228 - val_loss: 0.3883 - val_acc: 0.8353
Epoch 2/50
80000/80000 [==============================] - 1347s 17ms/step - loss: 0.4039 - acc: 0.8294 - val_loss: 0.3792 - val_acc: 0.8357
Epoch 3/50
80000/80000 [==============================] - 1373s 17ms/step - loss: 0.3954 - acc: 0.8330 - val_loss: 0.3720 - val_acc: 0.8417
Epoch 4/50
80000/80000 [==============================] - 1359s 17ms/step - loss: 0.3885 - acc: 0.8360 - val_loss: 0.3679 - val_acc: 0.8429
Epoch 5/50
80000/80000 [==============================] - 1370s 17ms/step - loss: 0.3829 - acc: 0.8375 - val_loss: 0.3629 - val_acc: 0.8435
Epoch 6/50
80000/80000 [==============================] - 1369s 17ms/step - loss: 0.3791 - acc: 0.8395 - val_loss: 0.3590 - val_acc: 0.8458
Epoch 7/50
80000/80000 [==============================] - 1928s 24ms/step - loss: 0.3751 - acc: 0.8411 - val_l

In [ ]:
score = fine_tuned.evaluate(xtest_npArray, ytest_npArray, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Predict using our Fine-tuned Model:

In [ ]:
#test_images, test_labels = next(test_batches)
#plots(test_images, title=test_labels) 

In [ ]:
#test_lables = test_lables[:,0]
#test_lables

In [ ]:
#predictions = fine_tuned.predict_generator(test_batches, steps =? , verbose = 0)

In [ ]:
#cm = confusion_matrix(test_labels, np.round(predictions[:,0]))

In [ ]:
#cm_plot_lables=['idc', nonidc']
#plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')